In [ ]:
import sys
sys.path.append('../..')

import os
import pandas as pd
import cudf
import numpy as np
import gquant.cuindicator as ci
from bqplot.traits import convert_to_date
import bqplot.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import datetime
import math
from bqplot import OHLC, LinearScale, DateScale, Axis, Figure, Bars, CATEGORY10, OrdinalScale, Lines, Tooltip
from bqplot.colorschemes import CATEGORY20
from gquant.dataframe_flow import run, load_workflow, viz_graph
import nxpd
from nxpd import draw

In [ ]:
node_csv = {"id": "node_csvdata",
            "type": "CsvStockLoader",
            "conf": {
                "path": "../data/stock_price_hist.csv.gz"                
            },
           "inputs": []}
node_asset = {"id": "node_assetFilter",
            "type": "AssetFilterNode",
            "conf": {
                "asset": 22123               
            },
           "inputs": ["node_csvdata"]}
node_sort = {"id": "node_sort",
            "type": "SortNode",
            "conf": {
                "keys": ['asset', 'datetime']
            },
            "inputs": ["node_csvdata"]
            }
node_stockSymbol = {"id": "node_stockSymbol",
            "type": "StockNameLoader",
            "conf": {
                "path": "../data/security_master.csv.gz"               
            },
           "inputs": []}    

In [ ]:
action = "load" if os.path.isfile('./.cache/node_csvdata.hdf5') else "save"
df = run([node_csv, node_sort], ['node_sort'], {'node_csvdata': {action: True}})[0]

def one_stock(df, stock_id):
    return df.query('asset==%s' % stock_id)

def slice_stock(df, year):
    beg_date = datetime.datetime.strptime(str(year)+'-01-01', '%Y-%m-%d')
    end_date = datetime.datetime.strptime(str(int(year)+1)+'-01-01', '%Y-%m-%d')
    return df.query('datetime<@end_date and datetime>=@beg_date')

indicator_lists = ['MA', 'EWA', 'Chaikin Oscillator', 'Average Directional Movement Index', 'MACD', 'TRIX', 'RSI',
                   'Bollinger Bands','Commodity Channel Index','Parabolic SAR','Rate of Change','Average True Range',
                   'Stochastic Oscillator D','Vortex Indicator','Mass Index','True Strength Index','Money Flow Index', 
                   'On Balance Volume','Force Index','Ease of Movement','Donchian Channel','Keltner Channel', 'Coppock Curve',
                  'Accumulation Distribution','Momentum','Ultimate Oscillator','Stochastic Oscillator K','KST Oscillator']
list_stocks = run([node_stockSymbol], ['node_stockSymbol'])[0].to_pandas().set_index('asset_name').to_dict()['asset']
main_figure_height='300px'
indicator_figure_height='150px'
figure_width = '1500px'

In [ ]:
add_stock_selector = widgets.Dropdown(options=list_stocks.keys(), value=None, description="Add stock")
year_selector = widgets.IntSlider(description="All Year", continuous_update=False)
year_selectors = []
def get_figure(selected, df):
    this_stock = one_stock(df, list_stocks[selected])
    this_stock_store = [this_stock]
    stock_selector = widgets.Dropdown(options=list_stocks.keys(), value=add_stock_selector.value, description="stock")
    indicator_selector = widgets.Dropdown(options=indicator_lists, value=None, description="Indicator")
    min_year = this_stock.datetime.to_array().min().astype(datetime.datetime).year
    max_year = this_stock.datetime.to_array().max().astype(datetime.datetime).year
    year_selector = widgets.IntSlider(min=min_year, max=max_year, description="Year", continuous_update=False)
    year = year_selector.value
    stock = slice_stock(this_stock, year)
    sc = LinearScale()
    sc2 = LinearScale()
    dt_scale = DateScale()
    ax_x = Axis(label='Date', scale=dt_scale)
    ax_y = Axis(label='Price', scale=sc, orientation='vertical', tick_format='0.0f')
    # Construct the marks
    ohlc = OHLC(x=stock.datetime, y=stock[['open','high','low', 'close']].as_gpu_matrix(), marker='candle', scales={'x': dt_scale, 'y': sc}, format='ohlc',
                stroke='blue', display_legend=True, labels=[selected])
    bar = Bars(x=stock.datetime, y=stock.volume, 
           scales={'x': dt_scale, 'y': sc2}, padding=0.2)
    def_tt = Tooltip(fields=['x', 'y'], formats=['%Y-%m-%d', '.2f'])
    bar.tooltip = def_tt
    bar.interactions = {
        'legend_hover': 'highlight_axes',
        'hover': 'tooltip', 
        'click': 'select',
    }
    sc.min = stock.close.min() - 0.3 * (stock.close.max() - stock.close.min()) 
    sc.max = stock.close.max()
    sc2.max = stock.volume.max()*4.0
    dt_scale.min = pd.Timestamp('%d-1-1' % year)
    dt_scale.max = pd.Timestamp('%d-1-1' % (year + 1))
    f = Figure(axes=[ax_x, ax_y], marks=[ohlc, bar], fig_margin={"top":0, "bottom":60, "left":60, "right":60})
    f.layout.height = main_figure_height
    f.layout.width = figure_width
    para_selectors = widgets.VBox([])
    color_id = [0]
    
    def update_graph(stock):
        with bar.hold_trait_notifications() as bc, ohlc.hold_trait_notifications() as oc:
            ohlc.y = stock[['open','high','low', 'close']].as_gpu_matrix()
            ohlc.x = stock.datetime
        
            bar.y = stock.volume
            bar.x = stock.datetime
    
            sc.min = stock.close.min() - 0.3 * (stock.close.max() - stock.close.min()) 
            sc.max = stock.close.max()
            sc2.max = stock.volume.max()*4.0
            dt_scale.min = pd.Timestamp('%d-1-1' % year_selector.value)
            dt_scale.max = pd.Timestamp('%d-1-1' % (year_selector.value + 1))
            update_range()
    
    def year_selection(*stock):
        stock = slice_stock(this_stock_store[0], year_selector.value)
        update_graph(stock)
        
    
    def stock_selection(*stock):
        this_stock_store[0] = one_stock(df, list_stocks[stock_selector.value])
        year_selector.min = this_stock_store[0].Dte.to_array().min().astype(datetime.datetime).year
        year_selector.max = this_stock_store[0].Dte.to_array().max().astype(datetime.datetime).year
        stock = slice_stock(this_stock_store[0], year_selector.value)
        ohlc.labels = [stock_selector.value]
        update_graph(stock)
        
    def update_figure(stock, objects):
        line = objects[0]
        with line.hold_trait_notifications():
            line.y = stock['out']
            line.x = stock.datetime
            
    def add_new_indicator(new_fig):
        # add new figure
        # take the axis from the fig
        empty = {"top":0, "bottom":0, "left":60, "right":60}
        axis_margin = {"top":0, "bottom":60, "left":60, "right":60}
        axes_copy = multiple_figs.children[-1].axes.copy() 
        multiple_figs.children[-1].fig_margin = empty
        to_be_removed = axes_copy[0]
        axes_copy.remove(to_be_removed)
        multiple_figs.children[-1].axes = axes_copy
        new_axes = new_fig.axes.copy()
        new_fig.axes = [to_be_removed] + new_axes
        new_fig.fig_margin = axis_margin
        # add new figure
        multiple_figs.children += (new_fig,)
        
    def indicator_selection(*stock):
        if indicator_selector.value is None:
            return
        color_id[0] = (color_id[0] + 1) % len(CATEGORY20)
        
        def setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun):

            with out:                    
                def update_df(para_selector_widgets):
                    my_stock = this_stock_store[0]
                    parameters = get_parameters(my_stock, para_selector_widgets)
                    output = indicator_fun(*parameters)
                    stock_df = process_outputs(output, my_stock)
                    stock = slice_stock(stock_df, year_selector.value)
                    return stock
                para_selector_widgets = get_para_widgets()   
                para_selectors.children += tuple(para_selector_widgets)                    
                stock = update_df(para_selector_widgets)
                figs = create_figure(stock)

                def update_para(*para):
                    stock = update_df(para_selector_widgets)
                    update_figure(stock, figs)
                    
                for selector in para_selector_widgets:
                    selector.observe(update_para, 'value')
                year_selector.observe(update_para, 'value')
                stock_selector.observe(update_para, 'value')
            
        if indicator_selector.value=='MA':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="avg periods")
                    para_selector_widgets = [para_selector]
                    
                    return para_selector_widgets

                def get_parameters(stock_df, para_selector_widgets):
                    return  (stock_df["close"],) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output, stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = stock_df['out'].fillna(math.inf)
                    return stock_df
                               
                def create_figure(stock):
                    line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc}, colors=[CATEGORY20[color_id[0]]])
                    figs = [line]
                    f.marks = f.marks + figs
                    return figs
                
                indicator_fun = ci.moving_average                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
        elif indicator_selector.value=='EWA':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="ewa avg periods")
                    para_selector_widgets = [para_selector]                    
                    return para_selector_widgets

                def get_parameters(stock_df, para_selector_widgets):
                    return  (stock_df["close"],) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output, stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = stock_df['out'].fillna(math.inf)
                    return stock_df
                               
                def create_figure(stock):
                    line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc}, colors=[CATEGORY20[color_id[0]]])
                    figs = [line]
                    f.marks = f.marks + figs
                    return figs
                
                indicator_fun = ci.exponential_moving_average                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
        elif indicator_selector.value=='Chaikin Oscillator':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntRangeSlider(value=[10, 30],
                                                           min=3,
                                                           max=60,
                                                           step=1,
                                                           description="Ch Oscillator:",
                                                           disabled=False,
                                                           continuous_update=False,
                                                           orientation='horizontal',
                                                           readout=True)
                    para_selector_widgets = [para_selector]                    
                    return para_selector_widgets

                def get_parameters(stock_df, para_selector_widgets):
                    widget = para_selector_widgets[0]
                    return  (stock_df["high"], stock_df["low"], stock_df["close"], stock_df["volume"], widget.value[0], widget.value[1])
                
                def process_outputs(output, stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = stock_df['out'].fillna(0)
                    return stock_df
                               
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Ch Osc', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)     
                    return figs
                     
                indicator_fun = ci.chaikin_oscillator                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
        elif indicator_selector.value=='Average Directional Movement Index':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntRangeSlider(value=[10, 30],
                                                           min=3,
                                                           max=60,
                                                           step=1,
                                                           description="ADMI:",
                                                           disabled=False,
                                                           continuous_update=False,
                                                           orientation='horizontal',
                                                           readout=True)
                    para_selector_widgets = [para_selector]                    
                    return para_selector_widgets

                def get_parameters(stock_df, para_selector_widgets):
                    widget = para_selector_widgets[0]
                    return  (stock_df["high"], stock_df["low"], stock_df["close"], widget.value[0], widget.value[1])
                
                def process_outputs(output, stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = stock_df['out'].fillna(0)
                    return stock_df
                               
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='ADMI', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)     
                    return figs
                     
                indicator_fun = ci.average_directional_movement_index                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)       
        elif indicator_selector.value=='MACD':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntRangeSlider(value=[10, 30],
                                                           min=3,
                                                           max=60,
                                                           step=1,
                                                           description="MACD:",
                                                           disabled=False,
                                                           continuous_update=False,
                                                           orientation='horizontal',
                                                           readout=True)
                    para_selector_widgets = [para_selector]                    
                    return para_selector_widgets

                def get_parameters(stock_df, para_selector_widgets):
                    widget = para_selector_widgets[0]
                    return  (stock_df["close"], widget.value[0], widget.value[1])
                
                def process_outputs(output, stock_df):
                    stock_df['out0'] = output.MACD
                    stock_df['out0'] = stock_df['out0'].fillna(0)
                    stock_df['out1'] = output.MACDsign
                    stock_df['out1'] = stock_df['out1'].fillna(0)
                    stock_df['out2'] = output.MACDdiff
                    stock_df['out2'] = stock_df['out2'].fillna(0)
                    return stock_df
                               
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='MACD', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=[stock['out0'], stock['out1'], stock['out2'] ], scales={'x': dt_scale, 'y': sc_co}) #
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)                    
                    return figs
                
                def up_figure(stock, objects):
                    line = objects[0]
                    with line.hold_trait_notifications():
                        line.y = [stock['out0'], stock['out1'], stock['out2']]
                        line.x = stock.datetime            
                     
                indicator_fun = ci.macd                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, up_figure, indicator_fun)
        elif indicator_selector.value=='TRIX':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="TRIX")
                    para_selector_widgets = [para_selector]                    
                    return para_selector_widgets

                def get_parameters(stock_df, para_selector_widgets):
                    return  (stock_df["close"],) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output, stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = stock_df['out'].fillna(0)
                    return stock_df
                               
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='TRIX', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs

                
                indicator_fun = ci.exponential_moving_average                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
        elif indicator_selector.value=='RSI':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="RSI")
                    para_selector_widgets = [para_selector]                    
                    return para_selector_widgets

                def get_parameters(stock_df, para_selector_widgets):
                    return  (stock_df["high"], stock_df["low"]) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output, stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = stock_df['out'].fillna(0)
                    return stock_df
                               
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='RSI', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs

                
                indicator_fun = ci.relative_strength_index                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                
        elif indicator_selector.value=='Bollinger Bands':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Bollinger Bands")
                    para_selector_widgets = [para_selector]                    
                    return para_selector_widgets

                def get_parameters(stock_df, para_selector_widgets):
                    return  (stock_df["close"],) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output, stock_df):
                    stock_df['out0'] = output.b1
                    stock_df['out0'] = stock_df['out0'].fillna(0)
                    stock_df['out1'] = output.b2
                    stock_df['out1'] = stock_df['out1'].fillna(0)
                    return stock_df
                               
                def create_figure(stock):
                    sc_co = LinearScale()
                    sc_co2 = LinearScale()
                    ax_y = Axis(label='Bollinger b1', scale=sc_co, orientation='vertical')
                    ax_y2 = Axis(label='Bollinger b2', scale=sc_co2, orientation='vertical', side='right')
                    new_line = Lines(x=stock.datetime, y=stock['out0'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]]) 
                    new_line2 = Lines(x=stock.datetime, y=stock['out1'], scales={'x': dt_scale, 'y': sc_co2}, colors=[CATEGORY20[(color_id[0] + 1) % len(CATEGORY20)]]) 
                    new_fig = Figure(marks=[new_line, new_line2], axes=[ax_y, ax_y2])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line, new_line2]
                    add_new_indicator(new_fig)
                    return figs
                
                def up_figure(stock, objects):
                    line = objects[0]
                    line2 = objects[1]
                    with line.hold_trait_notifications() as lc, line2.hold_trait_notifications() as lc2:
                        line.y = stock['out0']
                        line.x = stock.datetime        
                        line2.y = stock['out1']
                        line2.x = stock.datetime      
                
                indicator_fun = ci.bollinger_bands                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, up_figure, indicator_fun)
                
        elif indicator_selector.value=='Commodity Channel Index':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Commodity Channel Index")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    #print ((stock_df["high"],stock_df["low"], stock_df["close"]) + tuple([w.value for w in para_selector_widgets]))
                    return (stock_df["high"],stock_df["low"], stock_df["close"]) + tuple([w.value for w in para_selector_widgets])
                    
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='CCI', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.commodity_channel_index                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
       
        elif indicator_selector.value=='Parabolic SAR':
            with out:
                def get_para_widgets():
                    #para_selector = widgets.IntSlider(min=2, max=60, description="Parabolic SAR")
                    para_selector_widgets = []
                    return para_selector_widgets
                
                def get_parameters(stock_df,para_selector_widgets):
                    return (stock_df["high"], stock_df["low"], stock_df["close"]) 
                
                def process_outputs(output,stock_df):
                    stock_df['out0'] = output.PP
                    stock_df['out0'] = stock_df['out0'].fillna(0)
                    stock_df['out1'] = output.R1
                    stock_df['out1'] = stock_df['out1'].fillna(0)
                    stock_df['out2'] = output.S1
                    stock_df['out2'] = stock_df['out2'].fillna(0)
                    stock_df['out3'] = output.R2
                    stock_df['out3'] = stock_df['out3'].fillna(0)
                    stock_df['out4'] = output.S2
                    stock_df['out4'] = stock_df['out4'].fillna(0)
                    stock_df['out5'] = output.R3
                    stock_df['out5'] = stock_df['out5'].fillna(0)
                    stock_df['out6'] = output.S3
                    stock_df['out6'] = stock_df['out6'].fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    sc_co2 = LinearScale()
                    sc_co3 = LinearScale()
                    sc_co4 = LinearScale()
                    sc_co5 = LinearScale()
                    sc_co6 = LinearScale()
                    sc_co7 = LinearScale()
                    
                    ax_y = Axis(label='PPSR PP', scale=sc_co, orientation='vertical')
                    ax_y2 = Axis(label='PPSR R1', scale=sc_co2, orientation='vertical', side='right')
                    ax_y3 = Axis(label='PPSR S1', scale=sc_co3, orientation='vertical', side='right')
                    ax_y4 = Axis(label='PPSR R2', scale=sc_co4, orientation='vertical', side='right')
                    ax_y5 = Axis(label='PPSR S2', scale=sc_co5, orientation='vertical', side='right')
                    ax_y6 = Axis(label='PPSR R3', scale=sc_co6, orientation='vertical', side='right')
                    ax_y7 = Axis(label='PPSR S3', scale=sc_co7, orientation='vertical', side='right')
                    new_line = Lines(x=stock.datetime, y=stock['out0'], scales={'x': dt_scale, 'y': sc_co}, 
                                     colors=[CATEGORY20[color_id[0]]]) 
                    new_line2 = Lines(x=stock.datetime, y=stock['out1'], scales={'x': dt_scale, 'y': sc_co2}, 
                                      colors=[CATEGORY20[(color_id[0] + 1) % len(CATEGORY20)]]) 
                    new_line3 = Lines(x=stock.datetime, y=stock['out2'], scales={'x': dt_scale, 'y': sc_co3}, 
                                      colors=[CATEGORY20[(color_id[0] + 2) % len(CATEGORY20)]])
                    new_line4 = Lines(x=stock.datetime, y=stock['out3'], scales={'x': dt_scale, 'y': sc_co4}, 
                                      colors=[CATEGORY20[(color_id[0] + 3) % len(CATEGORY20)]]) 
                    new_line5 = Lines(x=stock.datetime, y=stock['out4'], scales={'x': dt_scale, 'y': sc_co5}, 
                                      colors=[CATEGORY20[(color_id[0] + 4) % len(CATEGORY20)]]) 
                    new_line6 = Lines(x=stock.datetime, y=stock['out5'], scales={'x': dt_scale, 'y': sc_co6}, 
                                      colors=[CATEGORY20[(color_id[0] + 5) % len(CATEGORY20)]]) 
                    new_line7 = Lines(x=stock.datetime, y=stock['out6'], scales={'x': dt_scale, 'y': sc_co7}, 
                                      colors=[CATEGORY20[(color_id[0] + 6) % len(CATEGORY20)]]) 
                
                    
                    new_fig = Figure(marks=[new_line, new_line2, new_line3, new_line4, 
                                            new_line5, new_line6, new_line7], 
                                     axes=[ax_y, ax_y2, ax_y3, ax_y4, ax_y5, ax_y6, ax_y7])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line, new_line2, new_line3, new_line4, new_line5, new_line6, new_line7]
                    add_new_indicator(new_fig)
                    return figs
                
                def up_figure(stock, objects):
                    line = objects[0]
                    line2 = objects[1]
                    with line.hold_trait_notifications() as lc, line2.hold_trait_notifications() as lc2:
                        line.y = stock['out0']
                        line.x = stock.datetime        
                        line2.y = stock['out1']
                        line2.x = stock.datetime      
                     
                indicator_fun = ci.ppsr                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, up_figure, indicator_fun)
                
                
        elif indicator_selector.value=='Rate of Change':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Rate of Change")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["close"],) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    print(stock_df['out'])
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Rate of Change', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.rate_of_change                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                    
        elif indicator_selector.value=='Average True Range':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Rate of Change")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["high"],stock_df["low"], stock_df["close"]) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Average True Range', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.average_true_range                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
            
        elif indicator_selector.value=='Stochastic Oscillator D':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Stochastic Oscillator D")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["high"],stock_df["low"], stock_df["close"]) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Stochastic Oscillator D', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.stochastic_oscillator_d                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                
        elif indicator_selector.value=='Vortex Indicator':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Vortex Indicator")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["high"],stock_df["low"], stock_df["close"]) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Vortex Indicator', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.vortex_indicator                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                
                
        elif indicator_selector.value=='Mass Index':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntRangeSlider(value=[10, 30],
                                                           min=3,
                                                           max=60,
                                                           step=1,
                                                           description="Mass Index:",
                                                           disabled=False,
                                                           continuous_update=False,
                                                           orientation='horizontal',
                                                           readout=True)
                    para_selector_widgets = [para_selector]                    
                    return para_selector_widgets

                def get_parameters(stock_df, para_selector_widgets):
                    widget = para_selector_widgets[0]
                    return  (stock_df["high"], stock_df["low"], widget.value[0], widget.value[1])
                
                def process_outputs(output, stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = stock_df['out'].fillna(0)
                    return stock_df
                               
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Mass Index', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)     
                    return figs
                     
                indicator_fun = ci.mass_index                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                
        elif indicator_selector.value=='True Strength Index':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntRangeSlider(value=[10, 30],
                                                           min=3,
                                                           max=60,
                                                           step=1,
                                                           description="True Strength Index",
                                                           disabled=False,
                                                           continuous_update=False,
                                                           orientation='horizontal',
                                                           readout=True)
                    para_selector_widgets = [para_selector]                    
                    return para_selector_widgets

                def get_parameters(stock_df, para_selector_widgets):
                    widget = para_selector_widgets[0]
                    return  (stock_df["close"],widget.value[0], widget.value[1])
                
                def process_outputs(output, stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = stock_df['out'].fillna(0)
                    return stock_df
                               
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='True Strength Index', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)     
                    return figs
                     
                indicator_fun = ci.true_strength_index                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                
        elif indicator_selector.value=='Money Flow Index':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Money Flow Index")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["high"],stock_df["low"], stock_df["close"], stock_df["volume"]) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Money Flow Index', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.money_flow_index                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
        
        
        elif indicator_selector.value=='On Balance volume':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="On Balance volume")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["close"], stock_df["volume"]) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='On Balance volume', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.on_balance_volume                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                
        
        elif indicator_selector.value=='Force Index':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Force Index")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["close"], stock_df["volume"]) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Force Index', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.force_index                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
        
        elif indicator_selector.value=='Ease of Movement':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Ease of Movement")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["high"], stock_df["low"], stock_df["volume"])
                + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Ease of Movement', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.ease_of_movement                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                
                
        elif indicator_selector.value=='Donchian Channel':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Donchian Channel")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["high"], stock_df["low"]) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Donchian Channel', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.donchian_channel                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                
                
        elif indicator_selector.value=='Keltner Channel':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Keltner Channel")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["high"], stock_df["low"], stock_df["close"]) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Keltner Channel', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.keltner_channel                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                
        elif indicator_selector.value=='Coppock Curve':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Coppock Curve")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["close"],) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Coppock Curve', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.coppock_curve                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                
                
        elif indicator_selector.value=='Accumulation Distribution':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Accumulation Distribution")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["high"], stock_df["low"], stock_df["close"], stock_df["volume"]) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Accumulation Distribution', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.accumulation_distribution                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
        
        elif indicator_selector.value=='Accumulation Distribution':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Accumulation Distribution")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["high"], stock_df["low"], stock_df["close"], stock_df["volume"]) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Accumulation Distribution', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.accumulation_distribution                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
         
        elif indicator_selector.value=='Momentum':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="Momentum")
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["close"],) + tuple([w.value for w in para_selector_widgets])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Momentum', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.momentum                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
          
        elif indicator_selector.value=='Ultimate Oscillator':
            with out:
                def get_para_widgets():
                    #para_selector = widgets.IntSlider(min=2, max=60, description="Ultimate Oscillator")
                    para_selector_widgets = []
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["high"], stock_df["low"], stock_df["close"]) 
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Ultimate Oscillator', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.ultimate_oscillator
                setup_indicator(get_para_widgets,get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
        
        
        elif indicator_selector.value=='Stochastic Oscillator K':
            with out:
                def get_para_widgets():
                    #para_selector = widgets.IntSlider(min=2, max=60, description="Ultimate Oscillator")
                    para_selector_widgets = []
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    return (stock_df["high"], stock_df["low"], stock_df["close"]) 
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='Stochastic Oscillator K', scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.stochastic_oscillator_k
                setup_indicator(get_para_widgets,get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
                
        
        
        elif indicator_selector.value=='KST Oscillator':
            with out:
                def get_para_widgets():
                    para_selector = widgets.IntSlider(min=2, max=60, description="KST Oscillator")
                    #para_selector_widgets = []
                    para_selector_widgets = [para_selector]
                    return para_selector_widgets
                
                def get_parameters(stock_df, para_selector_widgets):
                    param = [w.value for w in para_selector_widgets]
                    print (param)
                    param_grp = [param[0] + i for i in range(8)]
                    #param_grp = np.array(param_grp)
                    print (param_grp)
                    #return (stock_df["close"], 3,4,5,6,7,8,9,10)
                    return (stock_df["close"],param_grp[0],param_grp[1],param_grp[2],param_grp[3],
                            param_grp[4],param_grp[5],param_grp[6],param_grp[7])
                
                def process_outputs(output,stock_df):
                    stock_df['out'] = output
                    stock_df['out'] = output.fillna(0)
                    return stock_df
                
                def create_figure(stock):
                    sc_co = LinearScale()
                    ax_y = Axis(label='KST Oscillator',scale=sc_co, orientation='vertical')
                    new_line = Lines(x=stock.datetime, y=stock['out'], scales={'x': dt_scale, 'y': sc_co}, colors=[CATEGORY20[color_id[0]]])
                    new_fig = Figure(marks=[new_line], axes=[ax_y])
                    new_fig.layout.height = indicator_figure_height
                    new_fig.layout.width = figure_width                    
                    figs = [new_line]
                    # add new figure
                    add_new_indicator(new_fig)
                    return figs
                
                indicator_fun = ci.kst_oscillator                
                setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)
        indicator_selector.value=None
                
            
                
    year_selector.observe(year_selection, 'value')
    stock_selector.observe(stock_selection, 'value')
    indicator_selector.observe(indicator_selection, 'value')
    multiple_figs = widgets.VBox([f])
    return multiple_figs, year_selector, stock_selector, indicator_selector, para_selectors

#f, year_selector, stock_selector, indicator_selector, para_selectors = get_figure(add_stock_selector.value, df)

def stock_selection(*stock):
    if add_stock_selector.value is None:
        return
    f, year_selector, stock_selector, indicator_selector, para_selectors = get_figure(add_stock_selector.value, df)
    vbox = w.children[1]
    vbox.children += (widgets.HBox([f, widgets.VBox([year_selector, stock_selector, indicator_selector, para_selectors])]),) 
    year_selectors.append(year_selector)
    update_range()
    add_stock_selector.value = None

def update_range():
    min_vals = []
    max_vals = []
    for i in year_selectors:
        min_vals.append(i.min)
        max_vals.append(i.max)
    minV = max(min_vals)
    maxV = min(max_vals)
    if minV<=maxV:
        year_selector.disabled = False
        year_selector.max = maxV   
        year_selector.min = minV
    else:
        year_selector.disabled = True
    

out = widgets.Output(layout={'border': '1px solid black'})

def update_all_ranges(*arg):
    for i in year_selectors:
        i.value = year_selector.value
    
add_stock_selector.observe(stock_selection, 'value')
year_selector.observe(update_all_ranges, 'value')
selectors = widgets.HBox([add_stock_selector, year_selector])
w = widgets.VBox([selectors, widgets.VBox([])])
w

In [ ]:
output = ci.ppsr(df['high'],df['low'],df['close'])
print (output)
print(output.R1)
#print (output[100:].fillna(0))

In [ ]:
param_grp = [3 + i for i in range(8)]
print (param_grp)

In [ ]:
get_parameters(stock_df)

In [ ]:
out